# ETL Pipeline

## Extract Data

In [1]:
# %pip install -q tweepy
# %pip install -q python-decouple

In [2]:
# import necessay packages
import pandas as pd
import tweepy as tw
# from decouple import config

In [3]:
# config
consumer_key = 'ALS2SK37YuglyLYSyGGpfrbmY'
consumer_secret = '2Vyr3pBaHDsNaVX9IslmRHq25xR4zJjQjHpjUFTZzXj74rkvwS'
access_token = '1521183878576230400-4DzxTK7h5tI1gQ1f4QwoEWqfn0FNuV'
access_token_secret = 'sft4cbN41jMtVneJLD794rYRGJraHbb9ug4YELbZHFSyV'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAA6ScAEAAAAAnv1iKJAyg2INl1nA7%2BB0z%2FJbpK4%3DbrVth6B429DvCW0vsS4ufO6pLzmsAnhyYpaPRfWGoMWBKc6uRM'
user_id = '90676816'

In [4]:
# connect API
permission = tw.OAuthHandler(consumer_key, consumer_secret) # this object of ours will load the authorization request
permission.set_access_token(access_token, access_token_secret) # get ownership of our access object
client = tw.Client(bearer_token=bearer_token)
api = tw.API(permission) # use the API class, native to the library to generate the connection

In [5]:
query = 'Atletico-MG -filter:retweets OR Atlético Mineiro -filter:retweets OR #Galo -filter:retweets OR #ClubeAtléticoMineiro -filter:retweets'
galo_tweets = api.search_tweets(q=query, tweet_mode='extended', count=100)

id = []
created_at = []
full_text = []
favorite_count = []
retweet_count = []

for twitters in galo_tweets:
    id.append(twitters.id)
    created_at.append(twitters.created_at)
    full_text.append(twitters.full_text)
    favorite_count.append(twitters.favorite_count)
    retweet_count.append(twitters.retweet_count)

    # create dataframe
    df_final = {
      'tweet_id': id, 'created_at':created_at, 'text':full_text, 'likes':favorite_count, 
      'retweets':retweet_count
    }

In [6]:
search_tweets_df = pd.DataFrame(df_final)
search_tweets_df

,tweet_id,created_at,text,likes,retweets
0,1621806785949741056,2023-02-04 09:44:04+00:00,"Ipatinga x Atlético-MG: veja onde assistir, es...",18,0
1,1621698078485913600,2023-02-04 02:32:06+00:00,Ipatinga x Atlético-MG: veja informações e pro...,2,0
2,1621694494163836929,2023-02-04 02:17:51+00:00,04/02 - 19:00\n\n🖥PREMIERE \n🔴🟢 Ipatinga X Atl...,0,0
3,1621658306254356481,2023-02-03 23:54:03+00:00,Título e despedida: relembre um encontro marca...,0,0
4,1621657422741000192,2023-02-03 23:50:33+00:00,Título e despedida: relembre um encontro marca...,0,0
...,...,...,...,...,...
95,1619729429747433472,2023-01-29 16:09:23+00:00,"MINEIRO: Hulk marca no fim, Atlético-MG vence ...",0,0
96,1619728729818951682,2023-01-29 16:06:36+00:00,Campeonato Mineiro só serve para irritar preco...,625,27
97,1619728177982758914,2023-01-29 16:04:25+00:00,"Com gol de Hulk no finalzinho, Atlético-MG ven...",14,0
98,1619727924630011910,2023-01-29 16:03:24+00:00,"De virada!📣\nPaulinho marca, Alex Sandro empat...",43,4


In [7]:
# empty lists are created that will store the data extracted from the API call
id = []
date = []
text = []
retweets = []
answers = []
likes = []
mentions = []

# API call script with loop and pagination
for tweet in tw.Paginator(
    client.get_users_tweets, id=user_id, exclude=['retweets', 'replies'], #start_time = '2023-01-01T00:00:00Z',
    tweet_fields=['id', 'text', 'created_at', 'public_metrics'], max_results=100).flatten(limit=3200):

    # inserting the results in the lists
    id.append(tweet['id'])
    date.append(tweet['created_at'])
    text.append(tweet['text'])
    retweets.append(tweet['public_metrics']['retweet_count'])
    answers.append(tweet['public_metrics']['reply_count'])
    likes.append(tweet['public_metrics']['like_count'])
    mentions.append(tweet['public_metrics']['quote_count'])

    # create dataframe
    results = {
      'tweet_id':id, 'date':date, 'text':text, 'likes':likes, 
      'retweets':retweets, 'answers':answers, 'mentions':mentions
    }

In [8]:
paginator_df = pd.DataFrame(results)
paginator_df

,tweet_id,date,text,likes,retweets,answers,mentions
0,1621705351350595584,2023-02-04 03:01:00+00:00,💪🏾⚫⚪ HOJE TEM GAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,2749,561,140,73
1,1621671125695471623,2023-02-04 00:45:00+00:00,🗞 Guia da partida! Confira todas as informaçõe...,410,26,5,1
2,1621656026293600256,2023-02-03 23:45:00+00:00,💪🏽 O treino desta sexta-feira foi o último ant...,635,25,4,1
3,1621640927264612352,2023-02-03 22:45:00+00:00,💪🏽 Para seguir 100%: https://t.co/Pi5yIEFfhC\n...,601,27,4,1
4,1621631735745155074,2023-02-03 22:08:28+00:00,🏟️✊🏾 O @patrickChoco92 foi conhecer a nossa no...,797,44,11,1
...,...,...,...,...,...,...,...
820,1605168038277709824,2022-12-20 11:47:37+00:00,🎥 #GALOTVAOVIVO! O meio-campista Zaracho conce...,356,31,2,1
821,1605013867121319937,2022-12-20 01:35:00+00:00,🎥⚽️🎯🐔 https://t.co/UYZO8U3122,1578,97,18,12
822,1604994687093260288,2022-12-20 00:18:47+00:00,🎥 #GALOTVAOVIVO: Nesta terça-feira tem coletiv...,480,32,6,1
823,1604983668077785088,2022-12-19 23:35:00+00:00,💪🏽📸 Confira o álbum de fotos do treino de hoje...,893,54,5,2


In [9]:
test = pd.read_parquet('atleticomg-tweets_raw_own-twitters_atletico_own_tweets.parquet')
test

,tweet_id,date,text,likes,retweets,answers,mentions
0,1621671125695471623,2023-02-04 00:45:00+00:00,🗞 Guia da partida! Confira todas as informaçõe...,114,8,3,0
1,1621656026293600256,2023-02-03 23:45:00+00:00,💪🏽 O treino desta sexta-feira foi o último ant...,358,16,4,0
2,1621640927264612352,2023-02-03 22:45:00+00:00,💪🏽 Para seguir 100%: https://t.co/Pi5yIEFfhC\n...,393,17,4,0
3,1621631735745155074,2023-02-03 22:08:28+00:00,🏟️✊🏾 O @patrickChoco92 foi conhecer a nossa no...,535,31,6,0
4,1621603267640713216,2023-02-03 20:15:21+00:00,O @GaloDoido se inspirou em nosso novo camisa ...,318,18,5,1
...,...,...,...,...,...,...,...
820,1605013867121319937,2022-12-20 01:35:00+00:00,🎥⚽️🎯🐔 https://t.co/UYZO8U3122,1578,97,18,12
821,1604994687093260288,2022-12-20 00:18:47+00:00,🎥 #GALOTVAOVIVO: Nesta terça-feira tem coletiv...,480,32,6,1
822,1604983668077785088,2022-12-19 23:35:00+00:00,💪🏽📸 Confira o álbum de fotos do treino de hoje...,893,54,5,2
823,1604968569040683008,2022-12-19 22:35:00+00:00,🎥 Galo começa a calibrar a pontaria para 2023!...,910,63,16,8
